In [24]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats


In [25]:
filename = '../../outputs/correlation-05-17-2021_14-43-14.csv'
df = pd.read_csv(filename)
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)] ##Gets rid of infs, NaNs

In [30]:
over_90 = df[db.test_acc > 0.9]
x = "quality_L2"
y = "test_acc"
print(over_90.shape)
print(over_90.columns)

(485, 38)
Index(['model_num', 'test_acc', 'test_loss', 'train_acc', 'train_loss',
       'quality_L1', 'quality_L2', 'quality_prod', 'KG_L1', 'MC_L1', 'MC_L3',
       'ER_L1', 'mquality_L1', 'mquality_prod', 'mquality-wL1', 'mquality-wp',
       'qlayer0mode3', 'qlayer0mode4', 'qlayer-1mode3', 'qlayer-1mode4', 'KG0',
       'KG-1', 'MC0', 'MC-1', 'qnL1', 'qnL3', 'qnL4', 'qnL5', 'qnrL1', 'qnrL3',
       'qnrL4', 'qnrL5', 'qnprL1', 'qnprL3', 'qnprL4', 'qnprL5', 'qnrL6',
       'qnrL7'],
      dtype='object')
<ipython-input-30-d730135c49e7>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  over_90 = df[db.test_acc > 0.9]


In [36]:
cols = ['quality_L1', 'quality_L2', 'quality_prod', 'KG_L1', 'MC_L1', 'MC_L3',
       'ER_L1', 'mquality_L1', 'mquality_prod', 'mquality-wL1', 'mquality-wp',
       'qlayer0mode3', 'qlayer0mode4', 'qlayer-1mode3', 'qlayer-1mode4', 'KG0',
       'KG-1', 'MC0', 'MC-1', 'qnL1', 'qnL3', 'qnL4', 'qnL5', 'qnrL1', 'qnrL3',
       'qnrL4', 'qnrL5', 'qnprL1', 'qnprL3', 'qnprL4', 'qnprL5', 'qnrL6',
       'qnrL7']
test_acc_vec = over_90['test_acc']
train_acc_vec = over_90['train_acc']
results = {}

for x in cols:
    col = over_90[x]
    vec = col.to_numpy()
    results[x + "_test"] = stats.pearsonr(vec, test_acc_vec)[0]
    results[x + "_train"] = stats.pearsonr(vec, train_acc_vec)[0]

In [37]:
print(results)


{'quality_L1_test': (-0.0946713908368098, 0.03713976494315791), 'quality_L1_train': (-0.1686201913825031, 0.00019099521710648205), 'quality_L2_test': (-0.17500278868343933, 0.00010705120523815983), 'quality_L2_train': (-0.25686360304181366, 9.534814955503936e-09), 'quality_prod_test': (-0.036509932393578956, 0.4224137968449141), 'quality_prod_train': (-0.08951219906763837, 0.04881873018594128), 'KG_L1_test': (-0.008144572653479147, 0.8580117136615542), 'KG_L1_train': (-0.09789892418466978, 0.031113566072010237), 'MC_L1_test': (-0.012841365993408176, 0.7778793731993885), 'MC_L1_train': (0.04017796626765309, 0.37728961844241626), 'MC_L3_test': (0.044827298433764734, 0.3245421767551632), 'MC_L3_train': (0.027138301290806247, 0.5510237763523935), 'ER_L1_test': (0.08629305566686975, 0.05755826080679354), 'ER_L1_train': (0.062098859842328526, 0.1721353249332398), 'mquality_L1_test': (-0.49272347283102147, 4.977554597435411e-31), 'mquality_L1_train': (-0.48610383545075725, 3.948566013025388e-

In [1]:
plt.plot(over_90[x], over_90[y], '+', color='blue')
plt.xlabel(x)
plt.ylabel(y)
#plt.ylim([0,1.1])
plt.show()

NameError: name 'plt' is not defined